# Copied from BrA_Separation_merged-reproject on July 28, 2023

In [ ]:
from photutils.background import MMMBackground, MADStdBackgroundRMS
from astropy.modeling.fitting import LevMarLSQFitter

In [ ]:
from astropy import stats
from astropy.table import Table
from astropy.wcs import WCS

In [ ]:
from astropy.io import fits

In [ ]:
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.visualization import simple_norm
from astropy import wcs
from astropy import table
from astropy import units as u
import pylab as pl
pl.rcParams['figure.facecolor'] = 'w'
pl.rcParams['image.origin'] = 'lower'

In [ ]:
from astroquery.svo_fps import SvoFps

In [ ]:
import numpy as np

In [ ]:
import reproject

In [ ]:
from astropy import units as u

In [ ]:
basepath = '/orange/adamginsburg/jwst/brick/'

In [ ]:
fh_187 = fits.open(f'{basepath}/F187N/pipeline/jw02221-o001_t001_nircam_clear-f187n-merged_i2d.fits')
fh_182 = fits.open(f'{basepath}/F182M/pipeline/jw02221-o001_t001_nircam_clear-f182m-merged_i2d.fits')

In [ ]:
ww187 = wcs.WCS(fh_187['SCI'].header)
ww182 = wcs.WCS(fh_182['SCI'].header)

In [ ]:
instrument = fh_187[0].header['INSTRUME']
telescope = fh_187[0].header['TELESCOP']
filt187 = fh_187[0].header['FILTER']
wavelength_table_187 = SvoFps.get_transmission_data(f'{telescope}/{instrument}.{filt187}')
filt182 = fh_182[0].header['FILTER']
wavelength_table_182 = SvoFps.get_transmission_data(f'{telescope}/{instrument}.{filt182}')

In [ ]:
filt187, filt182

In [ ]:
waves_182 = wavelength_table_182['Wavelength']
trans_187 = np.interp(waves_182, wavelength_table_187['Wavelength'], wavelength_table_187['Transmission'])
trans_182 = wavelength_table_182['Transmission']

In [ ]:
pl.plot(waves_182, trans_182)
pl.plot(waves_182, trans_187)
pl.xlabel("Wavelength [Angstroms]")

In [ ]:
fractional_bandwidth_187 = (( (trans_182/trans_182.max()) *
                            (trans_187/trans_187.max()) ).sum()
                            / (trans_182/trans_182.max()).sum()
                           )
fractional_bandwidth_187

In [ ]:
fraction_bandwidth_empirical = 0.1742 # see end of document

In [ ]:
data_187_proj_182 = reproject.reproject_exact(fh_187['SCI'], fh_182['SCI'].header)

In [ ]:
from astropy.convolution import convolve_fft, Gaussian2DKernel

### from PSF analysis below, we need to smooth the ....
(This analysis was borrowed from BrA and isn't even correct there)

In [ ]:
cont182_sub_paa = (fh_182['SCI'].data - convolve_fft(data_187_proj_182[0]*fractional_bandwidth_187/(1-fractional_bandwidth_187),
                   Gaussian2DKernel(0.3)))
fits.PrimaryHDU(data=cont182_sub_paa, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F182_minus_F187_merged-reproject_theoretical_bandwidth.fits', overwrite=True)

In [ ]:
paa_minus_cont = data_187_proj_182[0] - cont182_sub_paa #* fractional_bandwidth_187
fits.PrimaryHDU(data=paa_minus_cont, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F187_minus_F182cont_merged-reproject_theoretical_bandwidth.fits', overwrite=True)

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(fh_182['SCI'].data, norm=simple_norm(cont182_sub_paa, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(cont182_sub_paa, norm=simple_norm(cont182_sub_paa, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(fh_187['SCI'].data, norm=simple_norm(cont182_sub_paa, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(paa_minus_cont, norm=simple_norm(paa_minus_cont, min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()

# PSF comparison

There was evidence of oversubtraction / undersubtraction in rings around the stars in the 187-182 image before we did PSF convolution:

In [ ]:
class slcgt:
    def __getitem__(self, args):
        return args

In [ ]:
dd = fh_182['SCI'].data
pl.figure(figsize=(12,8))
slc = slcgt()[1000:1250,1000:1250]
pl.imshow(dd[slc],
          norm=simple_norm(dd[slc], min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
slc = slcgt()[1000:1250,1000:1250]
pl.imshow(cont182_sub_paa[slc],
          norm=simple_norm(cont182_sub_paa[slc], min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
slc = slcgt()[1000:1250,1000:1250]
pl.imshow(paa_minus_cont[slc],
          norm=simple_norm(paa_minus_cont[slc], min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()

In [ ]:
import os
os.environ['WEBBPSF_PATH'] = '/orange/adamginsburg/jwst/webbpsf-data/'
import webbpsf
from webbpsf.utils import to_griddedpsfmodel

In [ ]:
import sys
sys.path.append(f'{basepath}/reduction/')

In [ ]:
import saturated_star_finding
import importlib as imp
imp.reload(saturated_star_finding)
from saturated_star_finding import get_psf

In [ ]:
psf_paa = get_psf(fh_187['SCI'].header)

In [ ]:
psf_182 = get_psf(fh_182['SCI'].header)

In [ ]:
from astropy.visualization import imshow_norm, simple_norm, LogStretch, AsinhStretch

In [ ]:
yy, xx = np.mgrid[-20:20.1:1,-20:20.1:1]
pl.figure(dpi=150)
ax = pl.subplot(1,3,1)
imshow_norm(psf_182(xx, yy), ax=ax, stretch=LogStretch())
ax = pl.subplot(1,3,2)
imshow_norm(psf_paa(xx, yy), ax=ax, stretch=LogStretch())
ax = pl.subplot(1,3,3)
imshow_norm(psf_paa(xx, yy) - psf_182(xx, yy), ax=ax, stretch=AsinhStretch())

In [ ]:
pb = psf_paa(xx, yy)
p4 = psf_182(xx, yy)
from astropy.convolution import convolve, convolve_fft, Gaussian2DKernel, AiryDisk2DKernel
def residual(x):
    return convolve(pb, Gaussian2DKernel(x)) - p4
def aresidual(x):
    return convolve(pb, AiryDisk2DKernel(x)) - p4

In [ ]:
pl.plot(np.linspace(0,0.6), [(residual(x)**2).sum() for x in np.linspace(0,0.6)])
pl.plot(np.linspace(0.1,0.4), [(residual(x)**2).sum() for x in np.linspace(0.1,0.4)])
pl.plot(np.linspace(0,0.6), [(aresidual(x)**2).sum() for x in np.linspace(0,0.6)])
pl.ylim(0,0.000003);

In [ ]:
resids = [(residual(x)**2).sum() for x in np.linspace(0.2,0.4)]
np.linspace(0.2,0.4)[np.argmin(resids)]

In [ ]:
yy, xx = np.mgrid[-20:20.1:1,-20:20.1:1]
pl.figure(dpi=150, figsize=(10,3.5))
ax = pl.subplot(1,3,1)
imshow_norm(psf_182(xx, yy), ax=ax, stretch=LogStretch())
ax = pl.subplot(1,3,2)
imshow_norm(psf_paa(xx, yy), ax=ax, stretch=LogStretch())
ax = pl.subplot(1,3,3)
im, norm = imshow_norm(convolve(psf_paa(xx, yy), Gaussian2DKernel(0.3)) - psf_182(xx, yy), ax=ax, stretch=AsinhStretch())
pl.colorbar(mappable=im)

In [ ]:
Gaussian2DKernel(0.3, x_size=xx.shape[1], y_size=yy.shape[0]).array.shape

### Attempt direct deconvolution

Maybe a Gaussian isn't the right kernel?

In [ ]:
ft4 = np.fft.fftshift(np.fft.fft2(psf_182(xx, yy)))
ftb = np.fft.fftshift(np.fft.fft2(psf_paa(xx, yy)))
ftk = np.fft.fftshift(np.fft.fft2(Gaussian2DKernel(0.3, x_size=xx.shape[1], y_size=yy.shape[0]).array))

In [ ]:
pl.figure(figsize=(10,3))
pl.subplot(1,4,1)
pl.imshow(np.abs(ftb))
pl.subplot(1,4,2)
pl.imshow(np.abs(ft4))
pl.subplot(1,4,3)
pl.imshow(np.abs(ftk))
pl.subplot(1,4,4)
pl.imshow(np.abs(ft4/ftb), vmin=0, vmax=1)

In [ ]:
#im, norm = imshow_norm(np.abs(ft4)/np.abs(ftb) * np.abs(ftb) > 1e-3, stretch=AsinhStretch(), vmax=1.1, vmin=0.9)
#pl.colorbar(mappable=im)
pl.subplot(1,2,1)
pl.plot(np.abs((ft4[20,:])))
pl.plot(np.abs((ftb[20,:])))
pl.subplot(1,2,2)
pl.plot(np.abs((ft4[20,:])) / np.abs((ftb[20,:])))
pl.plot(np.abs(ftk[20,:]) / ftk.max())
pl.ylim(0.96, 1.005);

In [ ]:
pl.plot((ft4[20,5:-5] / ftb[20,5:-5]))
pl.plot((ft4[5:-5,20] / ftb[5:-5,20]))
pl.plot(np.abs(ftk[5:-5,20])/np.abs(ftk).max())

In [ ]:
paa_to_182_kernel = np.abs((np.fft.fftshift(np.fft.fft2(ft4/ftb))))
paa_to_182_kernel /= paa_to_182_kernel.max()
imshow_norm(paa_to_182_kernel, stretch=LogStretch())
paa_to_182_kernel.max()
paa_to_182_kernel_ = np.zeros_like(paa_to_182_kernel)
paa_to_182_kernel_[15:25,15:25] = paa_to_182_kernel[15:25, 15:25]
paa_to_182_kernel = paa_to_182_kernel_

In [ ]:
yy, xx = np.mgrid[-20:20.1:1,-20:20.1:1]
pl.figure(dpi=150, figsize=(10,3.5))
ax = pl.subplot(1,3,1)
imshow_norm(psf_182(xx, yy), ax=ax, stretch=LogStretch())
ax = pl.subplot(1,3,2)
imshow_norm(convolve(psf_paa(xx, yy), Gaussian2DKernel(0.3, x_size=41, y_size=41)), ax=ax, stretch=LogStretch())
ax = pl.subplot(1,3,3)
im, norm = imshow_norm(convolve(psf_paa(xx, yy), Gaussian2DKernel(0.3, x_size=41, y_size=41)) - psf_182(xx, yy), ax=ax, stretch=AsinhStretch())
pl.colorbar(mappable=im)

### Kernel

The BrA needs to be convolved with an 0.8469 * 0.1 pixel = 0.085 pixel Gaussian

or an 0.3469 * 0.5 = 0.017 pixel gaussian!?

or an 0.30 pixel gaussian?!

In [ ]:
def residual2(x):
    return pb - convolve(p4, Gaussian2DKernel(x))
pl.plot(np.linspace(0,1), [(residual2(x)**2).sum() for x in np.linspace(0,1)])

# Further star removal via photometry

Starfinding on unsubtracted data so we can do some spatial cross-matching.

In [ ]:
stars_paa = DAOStarFinder(threshold=60, fwhm=2.302, peakmax=1e4)(fh_187['SCI'].data)
stars_paa['skycoord'] = ww187.pixel_to_world(stars_paa['xcentroid'], stars_paa['ycentroid'])
len(stars_paa)

In [ ]:
stars_182 = DAOStarFinder(threshold=10, fwhm=2.302, peakmax=900)(fh_182['SCI'].data)
stars_182['skycoord'] = ww182.pixel_to_world(stars_182['xcentroid'], stars_182['ycentroid'])
len(stars_182)

In [ ]:
matches, sep, _ = stars_paa['skycoord'].match_to_catalog_sky(stars_182['skycoord'], nthneighbor=1)

for cn in stars_paa.colnames:
    stars_paa.rename_column(cn, f"{cn}_187")
for cn in stars_182.colnames:
    stars_182.rename_column(cn, f"{cn}_182")
                         
stars_paa.add_column(name="sep_182_187", col=sep)
stars_paa.add_column(name="id_182_187", col=matches)
mergetbl = table.hstack([stars_paa, stars_182[matches]], join_type='exact')

In [ ]:
pl.hist(mergetbl['sep_182_187'].to(u.arcsec).value)

In [ ]:
radiff = (mergetbl['skycoord_187'].ra - mergetbl['skycoord_182'].ra).to(u.arcsec)
decdiff = (mergetbl['skycoord_187'].dec - mergetbl['skycoord_182'].dec).to(u.arcsec)
pl.scatter(radiff, decdiff, marker=',', s=1, alpha=0.8)
pl.axis([-0.5,0.5,-0.5,0.5])

# YIKES what the heck - why is there a big blob to the upper right?

In [ ]:
pl.figure(figsize=(10,8))
radiff = (mergetbl['skycoord_187'].ra - mergetbl['skycoord_182'].ra).to(u.arcsec)
decdiff = (mergetbl['skycoord_187'].dec - mergetbl['skycoord_182'].dec).to(u.arcsec)
topleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] > 1024)
pl.scatter(radiff[topleft], decdiff[topleft], marker=',', s=1, alpha=0.8)
topright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] > 1024)
pl.scatter(radiff[topright], decdiff[topright], marker=',', s=1, alpha=0.8)
bottomleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] < 1024)
pl.scatter(radiff[bottomleft], decdiff[bottomleft], marker=',', s=1, alpha=0.8)
bottomright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] < 1024)
pl.scatter(radiff[bottomright], decdiff[bottomright], marker=',', s=1, alpha=0.8)
pl.axis([-0.05,0.05,-0.05,0.05])
pl.xlabel("RA offset")
pl.ylabel("Dec offset")

In [ ]:
mergetbl['xcentroid_187'].max(), mergetbl['ycentroid_187'].max()

In [ ]:
dist_from_center = ((mergetbl['xcentroid_187'] - 1450)**2 + (mergetbl['ycentroid_187'] - 1145)**2)**0.5

In [ ]:
pl.plot(dist_from_center, mergetbl['sep_182_187'].to(u.arcsec).value, ',')
pl.plot(dist_from_center, dist_from_center/1750 * 0.045 + 0.008)
pl.plot(dist_from_center, dist_from_center/1750 * 0.035 - 0.004)
pl.ylim(0,0.07)
pl.xlabel("Distance from center of image (pixels)")
pl.ylabel("Offset from F182 (arcseconds)")

In [ ]:
ok = (mergetbl['sep_182_187'] < 0.05*u.arcsec)

In [ ]:
radiff = (mergetbl['skycoord_187'].ra - mergetbl['skycoord_182'].ra).to(u.arcsec)
decdiff = (mergetbl['skycoord_187'].dec - mergetbl['skycoord_182'].dec).to(u.arcsec)
pl.scatter(radiff[ok], decdiff[ok], marker=',', s=1, alpha=0.8)

In [ ]:
from astropy.wcs.utils import fit_wcs_from_points

In [ ]:
ww_187_refit = fit_wcs_from_points([mergetbl['xcentroid_187'][ok], mergetbl['ycentroid_187'][ok]], mergetbl['skycoord_182'][ok], sip_degree=1)

In [ ]:
ww_182_refit = fit_wcs_from_points([mergetbl['xcentroid_182'][ok], mergetbl['ycentroid_182'][ok]], mergetbl['skycoord_187'][ok], sip_degree=3)

In [ ]:
skycoord_182_refit = ww_182_refit.pixel_to_world(mergetbl['xcentroid_182'], mergetbl['ycentroid_182'])

In [ ]:
pl.figure(figsize=(14,9))
for sip_degree in (0,1,2,3):
    pl.subplot(2,2,sip_degree+1)
    
    ww_182_refit = fit_wcs_from_points([mergetbl['xcentroid_182'][ok], mergetbl['ycentroid_182'][ok]], mergetbl['skycoord_187'][ok], sip_degree=sip_degree)
    skycoord_182_refit = ww_182_refit.pixel_to_world(mergetbl['xcentroid_182'], mergetbl['ycentroid_182'])
    radiff = (mergetbl['skycoord_187'].ra - skycoord_182_refit.ra).to(u.arcsec)
    decdiff = (mergetbl['skycoord_187'].dec - skycoord_182_refit.dec).to(u.arcsec)
    sep = (radiff**2 + decdiff**2)**0.5
    ww_182_refit = fit_wcs_from_points([mergetbl['xcentroid_182'][ok & (sep < 0.01*u.arcsec)], mergetbl['ycentroid_182'][ok & (sep < 0.01*u.arcsec)]], mergetbl['skycoord_187'][ok & (sep < 0.01*u.arcsec)], sip_degree=sip_degree)
    skycoord_182_refit = ww_182_refit.pixel_to_world(mergetbl['xcentroid_182'], mergetbl['ycentroid_182'])
    radiff = (mergetbl['skycoord_187'].ra - skycoord_182_refit.ra).to(u.arcsec)
    decdiff = (mergetbl['skycoord_187'].dec - skycoord_182_refit.dec).to(u.arcsec)
    sep = (radiff**2 + decdiff**2)**0.5
    
    topleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] > 1024)
    sc = pl.scatter(radiff[topleft], decdiff[topleft], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[topleft]), np.median(decdiff[topleft]), marker='o', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    topright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] > 1024)
    sc = pl.scatter(radiff[topright], decdiff[topright], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[topright]), np.median(decdiff[topright]), marker='p', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    bottomleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] < 1024)
    sc = pl.scatter(radiff[bottomleft], decdiff[bottomleft], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[bottomleft]), np.median(decdiff[bottomleft]), marker='d', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    bottomright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] < 1024)
    sc = pl.scatter(radiff[bottomright], decdiff[bottomright], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[bottomright]), np.median(decdiff[bottomright]), marker='s', alpha=0.9, c=sc.get_facecolors(), zorder=25, edgecolors='k', s=60)
    pl.axis([-0.01,0.01,-0.01,0.01])

In [ ]:
pl.figure(figsize=(14,9))
for sip_degree in (0,1,2,3):
    pl.subplot(2,2,sip_degree+1)
    
    ww_187_refit = fit_wcs_from_points([mergetbl['xcentroid_187'][ok], mergetbl['ycentroid_187'][ok]], mergetbl['skycoord_182'][ok], sip_degree=sip_degree)
    skycoord_187_refit = ww_187_refit.pixel_to_world(mergetbl['xcentroid_187'], mergetbl['ycentroid_187'])
    radiff = (mergetbl['skycoord_182'].ra - skycoord_187_refit.ra).to(u.arcsec)
    decdiff = (mergetbl['skycoord_182'].dec - skycoord_187_refit.dec).to(u.arcsec)
    sep = (radiff**2 + decdiff**2)**0.5
    ww_187_refit = fit_wcs_from_points([mergetbl['xcentroid_187'][ok & (sep < 0.01*u.arcsec)], mergetbl['ycentroid_187'][ok & (sep < 0.01*u.arcsec)]], mergetbl['skycoord_182'][ok & (sep < 0.01*u.arcsec)], sip_degree=sip_degree)
    skycoord_187_refit = ww_187_refit.pixel_to_world(mergetbl['xcentroid_187'], mergetbl['ycentroid_187'])
    radiff = (mergetbl['skycoord_182'].ra - skycoord_187_refit.ra).to(u.arcsec)
    decdiff = (mergetbl['skycoord_182'].dec - skycoord_187_refit.dec).to(u.arcsec)
    sep = (radiff**2 + decdiff**2)**0.5
    
    topleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] > 1024)
    sc = pl.scatter(radiff[topleft], decdiff[topleft], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[topleft]), np.median(decdiff[topleft]), marker='o', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    topright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] > 1024)
    sc = pl.scatter(radiff[topright], decdiff[topright], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[topright]), np.median(decdiff[topright]), marker='p', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    bottomleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] < 1024)
    sc = pl.scatter(radiff[bottomleft], decdiff[bottomleft], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[bottomleft]), np.median(decdiff[bottomleft]), marker='d', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    bottomright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] < 1024)
    sc = pl.scatter(radiff[bottomright], decdiff[bottomright], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[bottomright]), np.median(decdiff[bottomright]), marker='s', alpha=0.9, c=sc.get_facecolors(), zorder=25, edgecolors='k', s=60)
    pl.axis([-0.01,0.01,-0.01,0.01])

In [ ]:
sip_degree = 3
ww_187_refit = fit_wcs_from_points([mergetbl['xcentroid_187'][ok], mergetbl['ycentroid_187'][ok]], mergetbl['skycoord_182'][ok], sip_degree=sip_degree)
skycoord_187_refit = ww_187_refit.pixel_to_world(mergetbl['xcentroid_187'], mergetbl['ycentroid_187'])
radiff = (mergetbl['skycoord_182'].ra - skycoord_187_refit.ra).to(u.arcsec)
decdiff = (mergetbl['skycoord_182'].dec - skycoord_187_refit.dec).to(u.arcsec)
sep = (radiff**2 + decdiff**2)**0.5
ww_187_refit = fit_wcs_from_points([mergetbl['xcentroid_187'][ok & (sep < 0.01*u.arcsec)], mergetbl['ycentroid_187'][ok & (sep < 0.01*u.arcsec)]], mergetbl['skycoord_182'][ok & (sep < 0.01*u.arcsec)], sip_degree=sip_degree)
skycoord_187_refit = ww_187_refit.pixel_to_world(mergetbl['xcentroid_187'], mergetbl['ycentroid_187'])
radiff = (mergetbl['skycoord_182'].ra - skycoord_187_refit.ra).to(u.arcsec)
decdiff = (mergetbl['skycoord_182'].dec - skycoord_187_refit.dec).to(u.arcsec)
sep = (radiff**2 + decdiff**2)**0.5


In [ ]:
ww_187_refit.to_header(relax=True).totextfile(f'{basepath}/reduction/headers/f187n_merged-reproject_refitted_to_f182m.hdr', overwrite=True)

In [ ]:
data_187_proj_182_refit, overlap_refit = reproject.reproject_exact((fh_187['SCI'].data, ww_187_refit), fh_182['SCI'].header)

In [ ]:
fits.PrimaryHDU(data=data_187_proj_182_refit, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F187_refitted187wcsto182_merged-reproject.fits', overwrite=True)

In [ ]:
cont182_sub_paa = fh_182['SCI'].data - data_187_proj_182_refit*fractional_bandwidth_187
fits.PrimaryHDU(data=cont182_sub_paa, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F182_minus_F187_refitted187wcsto182_merged-reproject.fits', overwrite=True)

In [ ]:
paa_minus_cont = data_187_proj_182_refit - cont182_sub_paa #* fractional_bandwidth_187
fits.PrimaryHDU(data=paa_minus_cont, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F187_minus_F182cont_refitted187wcsto182_merged-reproject.fits', overwrite=True)

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(cont182_sub_paa, norm=simple_norm(cont182_sub_paa, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(paa_minus_cont, norm=simple_norm(paa_minus_cont, min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(fh_187['SCI'].data, norm=simple_norm(fh_187['SCI'].data, min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar();

In [ ]:
latest_scalefactor = np.median(1/(mergetbl['flux_182'][ok]/mergetbl['flux_187'][ok]))
latest_scalefactor

# Goal is to remove stars as completely as possible
we are not trying to measure H-alpha excess here but remove the stars entirely from the image to reveal the extended emission

if we were trying instead to measure stellar halpha excess, we'd want to stick to theory and/or match on something else (maybe extended emission)

In [ ]:
xr = np.linspace(0,15)
pl.scatter(mergetbl['flux_187'][ok], mergetbl['flux_182'][ok], s=1, alpha=0.2)
pl.plot(xr, xr/fractional_bandwidth_187, color='red', label='Fractional Bandwidth')
scalefactor = 0.16
pl.plot(xr, xr/scalefactor, color='orange', label=f"Original Scalefactor = {scalefactor}")
post_destreak_scalefactor = 0.196
pl.plot(xr, xr/post_destreak_scalefactor, color='black', label=f"post-destreak scale factor={post_destreak_scalefactor}")
pl.plot(xr, xr/latest_scalefactor, color='green', label=f"'latest' scale factor={latest_scalefactor:0.3f}")
pl.legend(loc='best')
pl.xlabel("F187N")
pl.ylabel("F182M");
pl.xlim(0,20);

In [ ]:
pl.hist(1/(mergetbl['flux_182'][ok]/mergetbl['flux_187'][ok]), bins=np.linspace(0.05,0.25))
pl.axvline(scalefactor, color='orange')
pl.axvline(post_destreak_scalefactor, color='black')
pl.axvline(fractional_bandwidth_187, color='red')
pl.axvline(latest_scalefactor, color='green', linestyle=':')

In [ ]:
cont182_sub_paa = (fh_182['SCI'].data - convolve_fft(data_187_proj_182_refit*fractional_bandwidth_187,
                                                     Gaussian2DKernel(0.3))
                  )/(1-fractional_bandwidth_187)
#latest_scalefactor, 
fits.PrimaryHDU(data=cont182_sub_paa, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F182_minus_F187_refitted187wcsto182_merged-reproject.fits', overwrite=True)

In [ ]:
paa_minus_cont = data_187_proj_182_refit - cont182_sub_paa#convolve_fft(cont182_sub_paa, Gaussian2DKernel(0.5)) #* fractional_bandwidth_187
fits.PrimaryHDU(data=paa_minus_cont, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F187_minus_F182cont_refitted187wcsto182_merged-reproject.fits', overwrite=True)

Re-examine the images afer the empirical subtraction

In [ ]:
bx = 3035
by = 700
slc = slcgt()[by:by+250,bx:bx+250]

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(data_187_proj_182_refit[slc],
          norm=simple_norm(data_187_proj_182_refit[slc], min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()
pl.title("Zoom on PaA refit");

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(fh_187['SCI'].data[slc],
          norm=simple_norm(fh_187['SCI'].data[slc], min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()
pl.title("Zoom on PaA orig");

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(paa_minus_cont[slc],
          norm=simple_norm(paa_minus_cont[slc], min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()
pl.title("Zoom on PaA-cont");

In [ ]:
pl.figure(figsize=(12,8))
slc = slcgt()[1000:1250,1000:1250]
pl.imshow(cont182_sub_paa[slc],
          norm=simple_norm(cont182_sub_paa[slc], min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()
pl.title("Zoom on 182-PaA");

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(fh_182['SCI'].data [slc],
          norm=simple_norm(fh_182['SCI'].data [slc], min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()
pl.title("Zoom on 182");

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(data_187_proj_182_refit[slc],
          norm=simple_norm(data_187_proj_182_refit[slc], min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()
pl.title("Zoom on 187");

# Try to find the right scaling factor for the stars

The above scaling factor assumed that the stellar flux in the PaA band matches that in the F182M band, which is only true if most stars do not have PaA absorption.

In [ ]:
pl.scatter(fh_182['SCI'].data, data_187_proj_182_refit, s=1, alpha=0.005)
pl.plot([0,1000], [0,1000], color='k')
pl.plot([0,1000], [0,1000*fractional_bandwidth_187], color='r')
pl.axis([0,200,0,200]);
pl.title("If the data lay on the 1:1 line, then 100% of emission is PaA");

In [ ]:
pl.hist((data_187_proj_182_refit / fh_182['SCI'].data)[(fh_182['SCI'].data < 200) & (data_187_proj_182_refit < 100)],
        bins=np.linspace(0.1,4));
pl.title("The peak at 1.5x indicates that a large fraction of pixels are starry")